In [2]:
import time
import sys
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split, KFold
import multiprocessing
import os
from scipy import sparse
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score
from scipy.stats import spearmanr, pearsonr
from sklearn import ensemble
import datetime as dt
from datetime import date
import calendar
from calendar import weekday, day_name
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp
from patsy import dmatrix
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import scipy as sp
from sklearn.datasets import make_regression

In [4]:
train = pd.read_csv("data/train.csv", sep=',')
cols = ['store_nbr', 'item_nbr']
st_it = pd.DataFrame(train, columns=cols)
st_it

,store_nbr,item_nbr
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5
5,1,6
6,1,7
7,1,8
8,1,9
9,1,10


In [5]:
st_it.to_csv('data/store_item_nbrs.csv', sep=',', index=None)

In [6]:
import pandas as pd
import numpy as np

def create_vaild_item_store_combinations(_df):
    df = _df.copy()
    df['log1p'] = np.log(df['units'] + 1)
    
    g = df.groupby(["store_nbr", "item_nbr"])['log1p'].mean()
    g = g[g > 0.0]
    
    store_nbrs = g.index.get_level_values(0)
    item_nbrs = g.index.get_level_values(1)
    
    store_item_nbrs = sorted(zip(store_nbrs, item_nbrs), key = lambda t: t[1] * 10000 + t[0] )

    with open(store_item_nbrs_path, 'wb') as f: 
        f.write("store_nbr,item_nbr\n")
        for sno, ino in store_item_nbrs:
            f.write("{},{}\n".format(sno, ino))

store_item_nbrs_path = 'data/store_item_nbrs.csv'
df_train = pd.read_csv("data/train.csv")
create_vaild_item_store_combinations(df_train)

TypeError: a bytes-like object is required, not 'str'

In [ ]:
import pandas as pd
import numpy as np
import pickle

class SubmissionCreator(object):

    def create_id(self, row):
        date = row["date"]
        sno = row["store_nbr"]
        ino = row["item_nbr"]
        id = "{}_{}_{}".format(sno, ino, date)
        return id

    def create_id2(self, row):
        date = row["date"]
        s_no = row["store_nbr"]
        i_no = row["item_nbr"]
        id = str(i_no) + "_" + str(s_no) + "_" + date[0:4] + date[5:7] + date[8:10]
        return id

    def create_prediction_dict(self, fname_test, fname_p):
        d = dict()

        f_test = open(fname_test)
        f_p = open(fname_p)
        lines_test = f_test.readlines()
        lines_p = f_p.readlines()

        for line_test, line_p in zip(lines_test, lines_p):
            p_from_baseline = float(line_p.strip())

            I = line_test.strip().split("|")[-1]
            id2 = I.split(" ")[2]
            notsold = I.split(" ")[4]
            baseline = float(I.split(" ")[-1])

            if notsold == "True":
                pred = p_from_baseline + baseline
            else:
                pred = 0.0

            d[id2] = np.max([pred, 0.0])     

        return d  

    def create_submission(self, df_test, fname_submission):
        df = df_test

        fw = open(fname_submission, "w")
        fw.write("id,units\n") 

        for index, row in df.iterrows():
            id = self.create_id(row)
            id2 = self.create_id2(row)
            if prediction_dict.has_key(id2):
                log1p = prediction_dict[id2]
            else:
                log1p = 0.0
            units = np.exp(log1p) - 1   
            fw.write("{},{}\n".format(id, units))
            
        fw.close()
        print "finished {}".format(fname_submission)


submission_creator = SubmissionCreator()
df_test = pd.read_csv("data/test.csv")
prediction_dict = submission_creator.create_prediction_dict("model/vwdata_test.vwtxt", "model/vwdata.predict.txt")
submission_creator.create_submission(df_test, "submission/p.csv")

In [3]:
all_data_result = pd.read_csv('another_all_train_matrix.csv', sep=',')

In [ ]:
model = sm.OLS.from_formula( "np.log1p(units) ~ C(item_nbr) + C(weekday) + "
                                     "is_weekend + "
                                     "tmax + tmin + depart + dewpoint + wetbulb + sunrise + "
                                     "sunset + snowfall + preciptotal + stnpressure + sealevel + resultspeed + resultdir +"
                                     "avgspeed", data=all_data_result)
result = model.fit()

In [ ]:
dfX0 = pd.DataFrame(all_data_result, columns=boston.feature_names)
dfX = sm.add_constant(dfX0)
dfy = pd.DataFrame(boston.target, columns=["MEDV"])

dfy_predict = result.predict()

In [4]:
test = pd.read_csv('data/test.csv')
weather = pd.read_csv('data/weather_processed.csv')
key = pd.read_csv('data/key.csv')
holidays = get_holidays("data/holidays.txt")
holiday_names = get_holiday_names("data/holiday_names.txt")
test_key = pd.merge(test, key, on=['store_nbr'], how='left')

test_merge =  pd.merge(test_key, weather, on=['date', 'station_nbr'], how='left')
test_gg = preprocess(test_merge)
#test_gg.tail()

NameError: name 'get_holidays' is not defined

In [ ]:
model = sm.OLS.from_formula( "np.log1p(units) ~ C(item_nbr) + C(weekday) + C(is_weekend) + C(is_holiday_weekday) + C(is_holiday) +  C(holiday_name) + C(is_holiday_weekend) + C(around_BlackFriday) ", data=all_data_result)
result = model.fit()

In [ ]:
df = pd.DataFrame(result.predict(all_data_result))